In [1]:
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)

In [3]:
dir1

'/Users/davideposillipo/Documents/Lavoro/Alkemy/BRIO_x_Alkemy'

In [7]:
from brio.utils.Preprocessing import Preprocessing
from sklearn.model_selection import train_test_split
from pickle import dump, load
import pandas as pd
import numpy as np

from brio.bias.FreqVsFreqBiasDetector import FreqVsFreqBiasDetector

ModuleNotFoundError: No module named 'brio.utils'

## Importing Data and Trained Classifier

**UX**: 
- the user uploads the dataset with the features (X)
- the user uploads 
    - the sklearn model as pkl file **or**
    - a file with the predictions already created (Y). 
- the user uploads scaler and ohe (if needed)

If a model is provided, the application checks if the the provided datasets and model match in terms of column names. 

In [ ]:
input_data_path = "../data/raw_data/uci-default-of-credit-card/data/data.csv"
local_path_save = '../data/mlflow_artifacts/'

In [ ]:
fitted_ohe = load(open(local_path_save + '_ohe.pkl', 'rb')) 
fitted_scaler = load(open(local_path_save + '_scaler.pkl', 'rb'))

In [ ]:
pp = Preprocessing(input_data_path, "default")
X, Y = pp.read_dataframe()

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3, random_state=420)

X_test_ohe, _, _ = pp.preprocess_for_classification(df=X_test, 
                                                fit_ohe=True, 
                                                fitted_ohe=fitted_ohe,
                                                perform_scaling=True,
                                                fitted_scaler=fitted_scaler)

In [ ]:
with open("mlruns/1/1e4a0667c7a64cbe8c7b023410e5781c/artifacts/model/model.pkl", "rb") as file:
    classifier = load(file)

In [ ]:
predicted_prob = classifier.predict_proba(X_test_ohe)
predicted_values = classifier.predict(X_test_ohe)

#### Definition of conditioning variables
**UX**:
- the user selects the continuous features to be categorized and used for the creation of control groups. 

The application propose splitting cuts that make the resulting discrete distribution uniform (but also other binning procedures are possible). 

In [ ]:
def age_buckets(x):
    if x < 30:
        return 1
    elif x < 40:
        return 2
    else:
        return 3

X_test['age_buckets'] = X.x5_age.apply(age_buckets)

**UX**:
- the user selects from a drop down menu the variables to be used for the groups creation
- the user can select "check all the variables" and the application will use all the available discrete features

In [ ]:
conditioning_variables = ['x3_education', 'x4_marriage', 'age_buckets']

In [ ]:
df_with_predictions = pd.concat(
    [X_test.reset_index(drop=True), pd.Series(predicted_values)], axis=1).rename(columns={0:"predictions"})

## Bias Detection

**UX**:
- the user selects from a drop down menu the binary variable to be used for the distance calculation
- the user selects a threshold for the distance
- the user selects a Distance (if more than one are provided)

In [ ]:
bd = FreqVsFreqBiasDetector(distance="TVD")

**UX**:

The user has can select between two options:
- Option 1: distance between frequencies
- Option 2: distance from reference distribution
    - in this case, the user has to insert values for a reference distribution

## Option 1: Distance between frequencies

### Comparison of the two groups observed freqs (A1=high)

In [ ]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='x2_sex',
    threshold=0.1
)

In [ ]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='x2_sex'
)

In [ ]:
#distance = max( abs(female_0_freq - male_0_freq), abs(female_1_freq - male_1_freq) )

In [ ]:
df_with_predictions.groupby("x2_sex").predictions.value_counts(normalize=True)

### Comparison of the two groups observed freqs (A1=low)

In [ ]:
bd_low = FreqVsFreqBiasDetector(distance="TVD", A1="low")

In [ ]:
bd_low.compare_root_variable_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='x2_sex'
)

### Comparison of the two groups observed freqs, root_variable = age_buckets

In [ ]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='age_buckets',
    threshold=0.1
)

In [ ]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='age_buckets'
)

In [ ]:
bd_low.compare_root_variable_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='age_buckets'
)

### Comparison of the two groups observed freqs, root_variable = x3_education

In [ ]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='x3_education',
    threshold=0.1
)

In [ ]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='x3_education'
)

In [ ]:
bd_low.compare_root_variable_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='x3_education'
)

##### Using min as aggregating function

In [ ]:
bd_high_min = FreqVsFreqBiasDetector(distance="TVD", aggregating_function=min)
bd_low_min = FreqVsFreqBiasDetector(distance="TVD",aggregating_function=min, A1="low")

In [ ]:
bd_high_min.compare_root_variable_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='x3_education',
    threshold=0.1
)

In [ ]:
bd_high_min.compare_root_variable_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='x3_education'
)

In [ ]:
bd_low_min.compare_root_variable_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='x3_education'
)

##### Using std as aggregating function

In [ ]:
bd_high_std = FreqVsFreqBiasDetector(distance="TVD", aggregating_function=np.std)
bd_low_std = FreqVsFreqBiasDetector(distance="TVD", aggregating_function=np.std, A1="low")

In [ ]:
bd_high_std.compare_root_variable_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='x3_education',
    threshold=0.1
)

In [ ]:
bd_high_std.compare_root_variable_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='x3_education'
)

In [ ]:
bd_low_std.compare_root_variable_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='x3_education'
)

##### Using median as aggregating function

In [ ]:
bd_high_median = FreqVsFreqBiasDetector(distance="TVD", aggregating_function=np.median)
bd_low_median = FreqVsFreqBiasDetector(distance="TVD", aggregating_function=np.median, A1="low")

In [ ]:
bd_high_median.compare_root_variable_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='x3_education',
    threshold=0.1
)

In [ ]:
bd_high_median.compare_root_variable_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='x3_education'
)

In [ ]:
bd_low_median.compare_root_variable_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='x3_education'
)

##### Using mean as aggregating function

In [ ]:
bd_high_mean = FreqVsFreqBiasDetector(distance="TVD", aggregating_function=np.mean)
bd_low_mean = FreqVsFreqBiasDetector(distance="TVD", aggregating_function=np.mean, A1="low")

In [ ]:
bd_high_mean.compare_root_variable_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='x3_education',
    threshold=0.1
)

In [ ]:
bd_high_mean.compare_root_variable_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='x3_education'
)

In [ ]:
bd_low_mean.compare_root_variable_groups(
    dataframe=df_with_predictions,
    target_variable='predictions',
    root_variable='x3_education'
)